In [13]:
import requests
import json
from pprint import pprint

# Your configuration
url = "https://api.efadrin.io/api/v1/docsearch"
headers = {
    "X-Peelhunt-Token": "cda674de2e7aa9d79553660b1297e32a9bbf619b741020a896ee3d1700e5aa07"
}

payload = {
    "accountName": "FDRN_PEELHUNT",
    "searchText": "trading",
    "marketIds": "GB",
    "dateFrom": "2023-01-01",
    "dateTo": "2023-12-31",
    "searchTop": "1"
}

try:
    # Make request
    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"Error making request: {e}")

except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

except Exception as e:
    print(f"Unexpected error: {e}")

{
  "header": {
    "recordCount": 1,
    "statusCode": 0,
    "statusMsg": ""
  },
  "documents": [
    {
      "docGuid": "a7a39aa8-ba82-41c2-b7bf-da414bcdf422",
      "docId": 9876,
      "docType": {
        "id": 300
      },
      "rixmlType": "Company Note Long",
      "markets": [
        {
          "id": "GB"
        }
      ],
      "sectors": [
        {
          "id": 8
        }
      ],
      "industries": [
        {
          "id": 14
        }
      ],
      "authors": [
        {
          "id": 21623
        }
      ],
      "corps": [
        {
          "id": 517
        }
      ],
      "docTitle": "#TRENDING",
      "docSynopsis": "·  LBG is one of the UK’s largest social digital publishers, focused on a notoriously difficult demographic to reach – youths.|·  We address near-term headwinds from declining ad yields with conservatism in our forecasts, and steps that management have taken.|·  Going forward, we see many growth avenues, including building up a US Di

In [1]:
import requests
import json
from pprint import pprint

# Your configuration
base_url = "https://api.efadrin.io/api/"
endpoint = "v1/docretrieve"

request_url = base_url + endpoint

headers = {
    "X-Peelhunt-Token": "cda674de2e7aa9d79553660b1297e32a9bbf619b741020a896ee3d1700e5aa07"
}

payload = {
    "accountName": "FDRN_PEELHUNT",
    "docGuid":"6063300d-4452-43a2-9719-f041453cd5f8",
    "docType":"5"
}

try:
    # Make request
    response = requests.post(request_url, headers=headers, json=payload)
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"Error making request: {e}")

except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

except Exception as e:
    print(f"Unexpected error: {e}")

{
  "filename": "220707_Persimmon MN BC.xml",
  "docGuid": "6063300d-4452-43a2-9719-f041453cd5f8",
  "rixmlData": "<rixml:Research xmlns:rixml=\"http://www.rixml.org/2010/1/RIXML\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" createDateTime=\"2022-07-07T07:28:48+01:00\" language=\"eng\" researchID=\"6063300d-4452-43a2-9719-f041453cd5f8\"><rixml:Product eventIndicator=\"No\" productID=\"6063300d-4452-43a2-9719-f041453cd5f8\" sequence=\"1\"><rixml:StatusInfo currentStatusIndicator=\"Yes\" statusDateTime=\"2022-07-07T07:04:55Z\" statusType=\"Published\"><rixml:Version>3</rixml:Version></rixml:StatusInfo><rixml:Source><rixml:Organization primaryIndicator=\"Yes\" sequence=\"1\" type=\"Corporation\"><rixml:OrganizationID idType=\"EFA\">1400</rixml:OrganizationID><rixml:OrganizationName nameType=\"Display\">PEEL HUNT</rixml:OrganizationName><rixml:PersonGroup><rixml:PersonGroupMember primaryIndicator=\"Yes\" role=\"Author\" sequence=\"1\"><rixml:Person personID=\"Sam Cullen\"><rixm